In [1]:
file_path = r"C:\Users\20368750\OneDrive - Larsen & Toubro\Documents\Digital Initiative\Uservox_assignment\1.txt"

with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Print first few lines to verify structure
for line in lines[:10]:
    print(line.strip())


DATE                 NARRATION                                                           WITHDRAWAL (DR.)                   DEPOSIT (CR.)          BALANCE

01-Aug-2024          Opening Balance                                                                                                                      0

10-Aug-2024          DEPOSIT UPI CR-RRN:422394444743-From                                                                           100                  100
A/C:00000035399868304-IFSC:SBIN0013450-
DUM DUMMY M-Rem: Payment fr

20-Aug-2024          DEPOSIT UPI CR-RRN:423342557816-From                                                                            80                  180
A/C:915010056596069-IFSC:UTIB0004410-DUM


In [2]:
import re
import json

# Regular expression to detect dates (e.g., "10-Aug-2024")
date_pattern = re.compile(r"\d{2}-[A-Za-z]{3}-\d{4}")

transactions = []
current_transaction = {}

for line in lines:
    line = line.strip()
    
    # If the line contains a date, it's the start of a new transaction
    if date_pattern.match(line):
        # If there's an existing transaction, add it to the list before starting a new one
        if current_transaction:
            transactions.append(current_transaction)
        
        # Extract date
        date_match = date_pattern.search(line)
        if date_match:
            current_transaction = {
                "date": date_match.group(),
                "narration": "",
                "amount": 0,
                "type": ""
            }
    else:
        # Append to narration if it's a continuation
        if "narration" in current_transaction:
            current_transaction["narration"] += " " + line

    # Extract amounts (Debit and Credit)
    amount_match = re.findall(r"(\d+)", line)  # Extracts all numbers in the line
    
    if len(amount_match) == 2:  # This line contains an amount entry
        withdrawal = amount_match[0]
        deposit = amount_match[1]
        
        if withdrawal.isdigit():
            current_transaction["amount"] = int(withdrawal)
            current_transaction["type"] = "Debit"
        
        if deposit.isdigit():
            current_transaction["amount"] = int(deposit)
            current_transaction["type"] = "Credit"

# Append the last transaction
if current_transaction:
    transactions.append(current_transaction)

# Convert to JSON format and print output
print(json.dumps(transactions, indent=4))


[
    {
        "date": "01-Aug-2024",
        "narration": " ",
        "amount": 0,
        "type": ""
    },
    {
        "date": "10-Aug-2024",
        "narration": " A/C:00000035399868304-IFSC:SBIN0013450- DUM DUMMY M-Rem: Payment fr ",
        "amount": 13450,
        "type": "Credit"
    },
    {
        "date": "20-Aug-2024",
        "narration": " A/C:915010056596069-IFSC:UTIB0004410-DUM DUMDUM-Rem: Payment fr ",
        "amount": 4410,
        "type": "Credit"
    },
    {
        "date": "20-Aug-2024",
        "narration": " A/C:00000035588684559-IFSC:SBIN0013450- DUM DUM DUMDUM-Rem: UPI ",
        "amount": 13450,
        "type": "Credit"
    },
    {
        "date": "21-Aug-2024",
        "narration": " A/C:915010056596069-IFSC:UTIB0004410-DUMDUM DUM-Rem: Payment fr ",
        "amount": 4410,
        "type": "Credit"
    },
    {
        "date": "21-Aug-2024",
        "narration": " A/C:915010056596069-IFSC:UTIB0004410-DUMDUMDUM DUM-Rem: Payment fr ",
        "amount": 44

In [5]:
import re
import json

file_path = "1.txt"

# Regular expressions for detecting a date in the format "DD-MMM-YYYY"
date_pattern = re.compile(r"\d{2}-[A-Za-z]{3}-\d{4}")

# Read the file
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

transactions = []
current_transaction = None

for line in lines:
    line = line.strip()
    
    # If the line contains a date, it's a new transaction
    if date_pattern.match(line):
        # If we already captured a transaction, store it before processing the new one
        if current_transaction:
            transactions.append(current_transaction)
        
        # Start a new transaction
        date_match = date_pattern.search(line)
        current_transaction = {
            "date": date_match.group(), 
            "narration": "",
            "amount": 0,
            "type": ""
        }
    
    elif current_transaction:
        # Check if the line contains a transaction amount (debit or credit)
        amount_columns = re.findall(r"(\d+)", line)
        
        if len(amount_columns) == 2:  # Found debit and credit columns
            debit_amount = amount_columns[0]
            credit_amount = amount_columns[1]
            
            if debit_amount.isdigit() and credit_amount == "":  # Debit case
                current_transaction["amount"] = int(debit_amount)
                current_transaction["type"] = "Debit"
            
            if credit_amount.isdigit():  # Credit case
                current_transaction["amount"] = int(credit_amount)
                current_transaction["type"] = "Credit"
        
        else:
            # Add to narration if it's not an amount row
            current_transaction["narration"] += " " + line

# Append the last transaction
if current_transaction:
    transactions.append(current_transaction)

# Print the JSON output
print(json.dumps(transactions, indent=4))


[
    {
        "date": "01-Aug-2024",
        "narration": " ",
        "amount": 0,
        "type": ""
    },
    {
        "date": "10-Aug-2024",
        "narration": " DUM DUMMY M-Rem: Payment fr ",
        "amount": 13450,
        "type": "Credit"
    },
    {
        "date": "20-Aug-2024",
        "narration": " DUMDUM-Rem: Payment fr ",
        "amount": 4410,
        "type": "Credit"
    },
    {
        "date": "20-Aug-2024",
        "narration": " DUM DUM DUMDUM-Rem: UPI ",
        "amount": 13450,
        "type": "Credit"
    },
    {
        "date": "21-Aug-2024",
        "narration": " DUM-Rem: Payment fr ",
        "amount": 4410,
        "type": "Credit"
    },
    {
        "date": "21-Aug-2024",
        "narration": " DUM-Rem: Payment fr ",
        "amount": 4410,
        "type": "Credit"
    }
]


In [7]:
import pandas as pd
import re

file_path = "1.txt"

# Regular expression for date detection
date_pattern = re.compile(r"\d{2}-[A-Za-z]{3}-\d{4}")

# Read the file
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# List to store extracted transactions
transactions = []

# Temporary storage for each transaction
current_transaction = {"Date": "", "Narration": "", "Withdrawal (DR.)": "", "Deposit (CR.)": "", "Balance": ""}

for line in lines:
    line = line.strip()
    
    # Check if the line contains a new transaction date
    if date_pattern.match(line):
        # If a transaction was being built, append it before starting a new one
        if current_transaction["Date"]:
            transactions.append(current_transaction)
        
        # Extract the date and initialize a new transaction dictionary
        date_match = date_pattern.search(line)
        current_transaction = {
            "Date": date_match.group(),
            "Narration": "",
            "Withdrawal (DR.)": "",
            "Deposit (CR.)": "",
            "Balance": ""
        }
    
    elif current_transaction["Date"]:
        # Extract possible numeric values (for withdrawal, deposit, and balance)
        numbers = re.findall(r"(\d+)", line)

        if len(numbers) == 2:  # The line contains withdrawal & deposit columns
            current_transaction["Withdrawal (DR.)"] = numbers[0]
            current_transaction["Deposit (CR.)"] = numbers[1]
        
        elif len(numbers) == 1:  # The line contains only the balance
            current_transaction["Balance"] = numbers[0]
        
        else:
            # Append narration text
            current_transaction["Narration"] += " " + line

# Append the last transaction
if current_transaction["Date"]:
    transactions.append(current_transaction)

# Create a DataFrame
df = pd.DataFrame(transactions)

# Display the DataFrame
print(df)


          Date                      Narration   Withdrawal (DR.)  \
0  01-Aug-2024                                                     
1  10-Aug-2024   DUM DUMMY M-Rem: Payment fr   00000035399868304   
2  20-Aug-2024        DUMDUM-Rem: Payment fr     915010056596069   
3  20-Aug-2024       DUM DUM DUMDUM-Rem: UPI   00000035588684559   
4  21-Aug-2024           DUM-Rem: Payment fr     915010056596069   
5  21-Aug-2024           DUM-Rem: Payment fr     915010056596069   

  Deposit (CR.) Balance  
0                        
1       0013450          
2       0004410          
3       0013450          
4       0004410          
5       0004410          


In [10]:
import pandas as pd
import re

# Read the unstructured text data
# txt_file_path = "1.txt"

# # Read the content of the file
# with open(txt_file_path, "r", encoding="utf-8") as file:
#     txt_data = file.read()

txt_data = """   DATE                 NARRATION                                                           WITHDRAWAL (DR.)                   DEPOSIT (CR.)          BALANCE

    01-Aug-2024          Opening Balance                                                                                                                      0

    10-Aug-2024          DEPOSIT UPI CR-RRN:422394444743-From                                                                           100                  100
                         A/C:00000035399868304-IFSC:SBIN0013450-
                         DUM DUMMY M-Rem: Payment fr

    20-Aug-2024          DEPOSIT UPI CR-RRN:423342557816-From                                                                            80                  180
                         A/C:915010056596069-IFSC:UTIB0004410-DUM
                         DUMDUM-Rem: Payment fr

    20-Aug-2024          WTHDRL UPI DR-RRN:423355387656-To                                                       100                                         80
                         A/C:00000035588684559-IFSC:SBIN0013450-
                         DUM DUM DUMDUM-Rem: UPI

    21-Aug-2024          WTHDRL UPI DR-RRN:423461932796-To                                                         50                                        30
                         A/C:915010056596069-IFSC:UTIB0004410-DUMDUM
                         DUM-Rem: Payment fr

    21-Aug-2024          DEPOSIT UPI CR-RRN:423478998063-From                                                                           800                  830
                         A/C:915010056596069-IFSC:UTIB0004410-DUMDUMDUM
                         DUM-Rem: Payment fr
 """

# Regular expression to extract structured data
pattern = re.compile(r"(\d{2}-[A-Za-z]{3}-\d{4})\s+(.+?)\s+(\d+)?\s+(\d+)?\s+(\d+)", re.DOTALL)

# Extract data
transactions = []
for match in pattern.finditer(txt_data):
    date, narration, withdrawal, deposit, balance = match.groups()
    transactions.append([
        date.strip(),
        narration.strip(),
        float(withdrawal) if withdrawal else 0.0,
        float(deposit) if deposit else 0.0,
        float(balance)
    ])

# Convert to Pandas DataFrame
df = pd.DataFrame(transactions, columns=["Date", "Narration", "Withdrawal (DR)", "Deposit (CR)", "Balance"])

# Convert Date column to datetime format
df["Date"] = pd.to_datetime(df["Date"], format="%d-%b-%Y")

# Display DataFrame
df


,Date,Narration,Withdrawal (DR),Deposit (CR),Balance
0,2024-08-01,Opening Balance,0.0,0.0,10.0
1,2024-08-20,DEPOSIT UPI CR-RRN:423342557816-From,80.0,0.0,180.0
2,2024-08-20,WTHDRL UPI DR-RRN:423355387656-To,100.0,0.0,80.0
3,2024-08-21,WTHDRL UPI DR-RRN:423461932796-To,50.0,0.0,30.0
4,2024-08-21,DEPOSIT UPI CR-RRN:423478998063-From,800.0,0.0,830.0
